# Jupyter Notebook Testing

Jupyter Notebooks use cells that can easily be re ran and is very convenient for testing things out.
It's important to note that to be able to run these cells correctly, the dependencies need to be properly installed.
This means that the interpreter being used must point to the Python the one that is in your [venv](./.venv) created by poetry for this project.

**Note**: if you make changes to the source code, you will need to press `Restart` above to restart the kernel to ensure the changes occur here.
![image](../pictures/restartJupyterKernel.png)

In [1]:
import logging
logging.basicConfig(level=logging.INFO)
from backend.engine.types.IndustryType import IndustryType


tax_rates = {
    "corporate_income_tax": {itype.value: 0.0 for itype in IndustryType},
    "personal_income_tax": 0.0,
    "sales_tax": {itype.value: 0.0 for itype in IndustryType},
    "property_tax": 0.0,
    "tariffs": {itype.value: 0.0 for itype in IndustryType},
}

In [2]:
from backend.engine.interface.controller import ModelController
controller = ModelController()
model_id = controller.create_model(city_template=None, num_people=100,tax_rates=tax_rates)

In [7]:
controller.step_model(model_id,time=10)

INFO:MESA.mesa.model:calling model.step for timestep 41 
INFO:root:Determining price...NOT IMPLEMENTED
INFO:root:Tariff rate is 0.0
INFO:root:Determining price...NOT IMPLEMENTED
INFO:root:Tariff rate is 0.0
INFO:root:Determining price...NOT IMPLEMENTED
INFO:root:Tariff rate is 0.0
INFO:root:Determining price...NOT IMPLEMENTED
INFO:root:Tariff rate is 0.0
INFO:root:Determining price...NOT IMPLEMENTED
INFO:root:Tariff rate is 0.0
INFO:root:Determining price...NOT IMPLEMENTED
INFO:root:Tariff rate is 0.0
INFO:root:Determining price...NOT IMPLEMENTED
INFO:root:Tariff rate is 0.0
INFO:root:Determining price...NOT IMPLEMENTED
INFO:root:Tariff rate is 0.0
INFO:root:Determining price...NOT IMPLEMENTED
INFO:root:Tariff rate is 0.0
INFO:root:Determining price...NOT IMPLEMENTED
INFO:root:Tariff rate is 0.0
INFO:root:Determining price...NOT IMPLEMENTED
INFO:root:Tariff rate is 0.0
INFO:root:Determining price...NOT IMPLEMENTED
INFO:root:Tariff rate is 0.0
INFO:root:Determining price...NOT IMPLEMENT

In [8]:
indicators = controller.get_indicators(model_id, start_time=1, end_time=0, indicators=("GDP", "Unemployment"))
print(indicators)

    GDP  Unemployment
1     0           1.0
2     0           1.0
3     0           1.0
4     0           1.0
5     0           1.0
6     0           1.0
7     0           1.0
8     0           1.0
9     0           1.0
10    0           1.0
11    0           1.0
12    0           1.0
13    0           1.0
14    0           1.0
15    0           1.0
16    0           1.0
17    0           1.0
18    0           1.0
19    0           1.0
20    0           1.0
21    0           1.0
22    0           1.0
23    0           1.0
24    0           1.0
25    0           1.0
26    0           1.0
27    0           1.0
28    0           1.0
29    0           1.0
30    0           1.0
31    0           1.0
32    0           1.0
33    0           1.0
34    0           1.0
35    0           1.0
36    0           1.0
37    0           1.0
38    0           1.0
39    0           1.0
40    0           1.0
41    0           1.0
42    0           1.0
43    0           1.0
44    0           1.0
45    0   